In [1]:
%reload_ext autoreload
%autoreload 2

In [18]:
import pandas as pd
import sys
import os
sys.path.append(os.path.abspath('../'))
from src.hopsworks_connections import pull_data, upload_data

# Print the current working directory
current_working_directory = os.getcwd()
print("Current Working Directory:", current_working_directory)

Current Working Directory: /Users/davydsadovskyy/GitBackedShit/crypto-prediction/notebooks


### Get a dataframe of the transformed ethereum OHLC data we made in 02_ohlc_feature_target_engineering.ipynb

In [21]:
eth_ohlc = pd.read_csv('../data/transformed/ohlc/ethereum.csv')
eth_ohlc

,date,open,high,low,close,volume,volume_eth,market_cap,tmw_avg_high_close,tmw_percent_increase,...,october,november,december,monday,tuesday,wednesday,thursday,friday,saturday,sunday
0,2015-11-15,0.8912,0.9215,0.8750,0.9064,4.118000e+05,458365.0,6.720000e+07,0.93685,0.032503,...,0,1,0,0,0,0,0,0,0,1
1,2015-11-16,0.9062,0.9447,0.8920,0.9290,6.209000e+05,676442.0,6.860000e+07,1.02000,0.089216,...,0,1,0,1,0,0,0,0,0,0
2,2015-11-17,0.9249,1.0300,0.9058,1.0100,1.100000e+06,1183690.0,7.220000e+07,1.00000,-0.010000,...,0,1,0,0,1,0,0,0,0,0
3,2015-11-18,0.9900,1.0100,0.9405,0.9900,6.811000e+05,691994.0,7.360000e+07,0.98275,-0.007377,...,0,1,0,0,0,1,0,0,0,0
4,2015-11-19,0.9887,1.0100,0.9375,0.9555,4.435000e+05,455866.0,7.280000e+07,0.94030,-0.016165,...,0,1,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3056,2024-04-16,3098.0000,3121.0000,3003.0000,3082.0000,1.710000e+10,5557389.0,3.757000e+11,3051.50000,-0.009995,...,0,0,0,0,1,0,0,0,0,0
3057,2024-04-17,3083.0000,3119.0000,2926.0000,2984.0000,1.550000e+10,5093979.0,3.722000e+11,3075.50000,0.029751,...,0,0,0,0,0,1,0,0,0,0
3058,2024-04-18,2986.0000,3087.0000,2960.0000,3064.0000,1.350000e+10,4453302.0,3.706000e+11,3101.00000,0.011932,...,0,0,0,0,0,0,1,0,0,0
3059,2024-04-19,3061.0000,3117.0000,2879.0000,3085.0000,1.740000e+10,5675824.0,3.746000e+11,3160.50000,0.023889,...,0,0,0,0,0,0,0,1,0,0


### for simplicity, we will also later be adding next day predictions to this same hopsworks feature group. Let's define prediction columns for each of the response variables.

In [22]:
# Convert response binary columns to nullable integer type
columns = [
    'pred_tmw_positive_percent_increase_binary',
    'pred_tmw_1_0_percent_increase_binary',
    'pred_tmw_1_25_percent_increase_binary',
    'pred_tmw_1_5_percent_increase_binary',
    'pred_tmw_1_75_percent_increase_binary',
    'pred_tmw_2_0_percent_increase_binary',
    'pred_tmw_2_25_percent_increase_binary',
    'pred_tmw_2_5_percent_increase_binary',
    'pred_tmw_2_75_percent_increase_binary',
    'pred_tmw_3_0_percent_increase_binary'
]
for column in columns:
    eth_ohlc[column] = pd.Series([pd.NA] * len(eth_ohlc)).astype('Int64')

In [20]:
eth_ohlc.dtypes[15:25]

tmw_2_25_percent_increase_binary          float64
tmw_2_5_percent_increase_binary           float64
tmw_2_75_percent_increase_binary          float64
tmw_3_0_percent_increase_binary           float64
tmw_positive_percent_increase_binary      float64
ema_2                                     float64
rsi_2                                     float64
sma_2                                     float64
last_2_day_1_0_percent_increase_count     float64
last_2_day_1_25_percent_increase_count    float64
dtype: object

### Upload this df to hopsworks feature store

In [25]:
# hopsworks requires datetime, when reading back from .csv, the datetime designation apparently goes away, along with nonnullable ints
eth_ohlc['date'] = pd.to_datetime(eth_ohlc['date'])

# Convert response binary columns to nullable integer type
columns = [
    'tmw_positive_percent_increase_binary',
    'tmw_1_0_percent_increase_binary',
    'tmw_1_25_percent_increase_binary',
    'tmw_1_5_percent_increase_binary',
    'tmw_1_75_percent_increase_binary',
    'tmw_2_0_percent_increase_binary',
    'tmw_2_25_percent_increase_binary',
    'tmw_2_5_percent_increase_binary',
    'tmw_2_75_percent_increase_binary',
    'tmw_3_0_percent_increase_binary'
]
for column in columns:
    eth_ohlc[column] = eth_ohlc[column].astype('Int64')

upload_data(eth_ohlc, 'ethereum_ohlc', 1)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (3.7) by running 'pip install hopsworks==3.7.*'



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/582805
Connected. Call `.close()` to terminate connection gracefully.
Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/582805/fs/578628/fg/743754


Uploading Dataframe: 100.00% |██████████| Rows 3061/3061 | Elapsed Time: 00:03 | Remaining Time: 00:00

Launching job: ethereum_ohlc_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/582805/jobs/named/ethereum_ohlc_1_offline_fg_materialization/executions


### Pull from the hopsworks to check if the df is fine - theres a several minute lag time between uploading this data and being able to pull it...

In [26]:
hopsworks_eth_ohlc = pull_data('ethereum_ohlc', 1, 'ethereum_ohlc', 1)
hopsworks_eth_ohlc

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/582805
Connected. Call `.close()` to terminate connection gracefully.
Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/582805/fs/578628/fv/ethereum_ohlc/version/1
Error: Reading data from Hopsworks, using ArrowFlight           


FeatureStoreException: Could not read data using ArrowFlight. If the issue persists, use read_options={"use_hive": True} instead.